<a href="https://colab.research.google.com/github/CarsonHhs/InventoryManagement/blob/main/OilPrice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [114]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import math

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM

In [292]:
df = pd.read_csv('/content/drive/MyDrive/Colab/Price.csv') #,parse_dates= ['Date'],dayfirst=True)
#df.set_index('Date', inplace=True)
#df =df.resample('m').sum()
#df.reset_index(inplace=True)

In [293]:
df['Date'] = pd.to_datetime(df['Date'])
#df = df.reindex(index= df.index[::-1])

In [298]:
#zaman = np.arange(1, len(df) + 1, 1)
OHCL_avg = df.mean(axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning:

DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.



In [299]:
OHCL_avg = np.reshape(OHCL_avg.values, (len(OHCL_avg),1)) 

In [300]:
scaler = MinMaxScaler(feature_range=(0,1))
OHCL_avg = scaler.fit_transform(OHCL_avg)

In [301]:
#print(OHCL_avg)
train_OHLC = int(len(OHCL_avg)*0.9)
test_OHLC =  int(len(OHCL_avg)*0.1)

print(train_OHLC,test_OHLC)

train_OHLC, test_OHLC = OHCL_avg[0:train_OHLC,:], OHCL_avg[train_OHLC:len(OHCL_avg),:]

381 42


In [302]:
#output = activation(dot(input, kernel) + bias)

def new_dataset(dataset, step_size):
    data_X, data_Y = [], []
    for i in range(len(dataset)-step_size-1):
        a = dataset[i:(i+step_size), 0]
        data_X.append(a)
        data_Y.append(dataset[i + step_size, 0])
    return np.array(data_X), np.array(data_Y)

In [303]:
step_size = 4

In [305]:
#Train the datasets and test it
trainX, trainY = new_dataset(train_OHLC,step_size)
testX, testY = new_dataset(test_OHLC,step_size)

In [306]:
trainX = np.reshape(trainX, (trainX.shape[0],1,trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0],1,testX.shape[1]))

In [307]:
model = Sequential()
model.add(LSTM(128, input_shape=(1, step_size)))
model.add(Dropout(0.1))
model.add(Dense(1))
model.add(Activation('linear'))

In [308]:
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=10, batch_size=25, verbose=2)

Epoch 1/10
16/16 - 2s - loss: 0.0986
Epoch 2/10
16/16 - 0s - loss: 0.0380
Epoch 3/10
16/16 - 0s - loss: 0.0148
Epoch 4/10
16/16 - 0s - loss: 0.0107
Epoch 5/10
16/16 - 0s - loss: 0.0079
Epoch 6/10
16/16 - 0s - loss: 0.0056
Epoch 7/10
16/16 - 0s - loss: 0.0049
Epoch 8/10
16/16 - 0s - loss: 0.0043
Epoch 9/10
16/16 - 0s - loss: 0.0040
Epoch 10/10
16/16 - 0s - loss: 0.0039


In [309]:
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)


trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])

testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])


In [310]:
print(trainPredict.shape, testPredict.shape)

(376, 1) (38, 1)


In [314]:
a = np.empty((step_size+1,))
a [:]= np.nan
b = np.empty((int(testY.size+ step_size+1),))
b [:]= np.nan
c=trainPredict.reshape(trainY.size)
d = np.concatenate((a, c))
d= np.concatenate((d,b))
df['Train'] = d

In [317]:
a = np.empty((trainY.size+step_size*2+2,))
a [:]= np.nan
b=testPredict.reshape(testY.size,)
c = np.concatenate((a,b))
df['Test'] = c

In [ ]:
!pip install "notebook>=5.3" "ipywidgets>=7.5"

In [318]:
df.tail(10)

,Date,Price,Train,Test
414,2020-07-01,40.71,NaN,29.977591
415,2020-08-01,42.34,NaN,28.395618
416,2020-09-01,39.63,NaN,30.599951
417,2020-10-01,39.40,NaN,35.743584
418,2020-11-01,40.94,NaN,37.737041
419,2020-12-01,47.02,NaN,37.940418
420,2021-01-01,52.00,NaN,38.149601
421,2021-02-01,59.04,NaN,39.577553
422,2021-03-01,62.33,NaN,42.452290
423,2021-04-01,61.72,NaN,46.986217


In [322]:
import plotly.graph_objects as go

# Create figure
fig = go.Figure()

fig.add_trace(
    go.Scatter(x=list(df.Date), y=list(df.Price), line=dict(color="green"), name= 'price')
    )

fig.add_trace(
    go.Scatter(x=list(df.Date), y=list(df.Train), line=dict(color="red"), name = 'Train')
    )

fig.add_trace(
    go.Scatter(x=list(df.Date), y=list(df.Test), line=dict(color="blue"), name = 'Test')
    )


# Set title
fig.update_layout(
    title_text="Time series with range slider and selectors"
)



In [321]:
# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.show()